## Prefer exceptions to returning None
Returning None from a function is error prone. Two ways to reduce:
* Split the return value into a two-tuple = (successOrFailure, result). The problem is that callers can just ignore the first part of this tuple.
* The second, better way to reduce these errors is to never return None at all.

## Know how closures interact with variable scope


## Consider generators instead of returning lists

In [24]:
def func1(someList):
    return [x**2 for x in someList]

func1(range(11))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [25]:
(x**2 for x in range(11))

<generator object <genexpr> at 0x0000000004233EA0>

In [26]:
list((x**2 for x in range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [28]:
def func2(someList):
    return (x**2 for x in someList)
    
it = func2(range(11))
print(it)
print(list(it))

<generator object <genexpr> at 0x000000000423C168>
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


Each value passed to yield by the generator will be returned by the iterator to the caller.

In [30]:
def func3(someList):
    for x in someList:
        if x % 3 == 0:
            yield x**2
            
it = func3(range(11))
print(it)
print(list(it))

<generator object func3 at 0x000000000423C438>
[0, 9, 36, 81]


Lets test out the speed

In [34]:
def funcOne(someList):
    return [x**2 for x in someList if x % 3 == 0]

def funcTwo(someList):
    for x in someList:
        if x % 3 == 0:
            yield x**2

In [36]:
funcOne(range(21))

[0, 9, 36, 81, 144, 225, 324]

In [39]:
list(funcTwo(range(21)))

[0, 9, 36, 81, 144, 225, 324]

In [44]:
%timeit funcOne(range(101))

100000 loops, best of 3: 7.28 µs per loop


In [45]:
%timeit funcTwo(range(101))

1000000 loops, best of 3: 1.05 µs per loop


Looks like we get a decent speed boost in addition to memory efficiencies.

## Be defensive when iterating over arguments
If we loop over generators the output gets exhausted after the first loop (as seen above). If we want to loop over the results more than once then it'd be best to store the generator results in a list. However, attempting to store a very large list in memory could cause the program to crash, in which case get a function to return a new iterator as required. An example:

In [60]:
# this is actually quite slow
class SomeIter():
    def __init__(self, someList):
        self._someList = someList
        self._index = 0
        
    def __iter__(self):
        return self
    
    def next(self): # __next__ in python 3
        try:
            x = self._someList[self._index]
        except IndexError:
            raise StopIteration
        self._index += 1
        return result ** 2
    

In [61]:
list(SomeIter(range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [64]:
%timeit list(SomeIter(range(51)))

10000 loops, best of 3: 22 µs per loop


In [65]:
%timeit [x**2 for x in range(51)]

100000 loops, best of 3: 3.47 µs per loop


In [68]:
class MoreIter():
    def __init__(self, someList):
        self._someList = someList
        
    def getIterable(self):
        for x in self._someList:
            yield x**2
            
list(MoreIter(range(11)).getIterable())

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [69]:
%timeit list(MoreIter(range(51)).getIterable())

100000 loops, best of 3: 5.22 µs per loop


There is overhead time in creating the class instance, but MoreIter is faster than SomeIter as we are working with a generator rather than a list.

In [75]:
getGenerator = lambda someList: (x**2 for x in someList)
print(list(getGenerator(range(11))))
print(list(getGenerator(range(11))))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


## Reduce visual noise with variable positional arguments

In [1]:
def sumValues(*args): # a very basic example
    valueSum = 0
    for arg in args:
        valueSum += arg
    return valueSum        

In [2]:
sumValues(1, 2)

3

In [3]:
sumValues(1, 2, 3)

6

In [4]:
sumValues(1, 2, 3, 4)

10

## Provide optional behavior with keyword arguments

In [5]:
def sumValues(*args, **kwargs): # another very basic example
    valueSum = 0
    for arg in args:
        valueSum += arg
        
    if "printSum" in kwargs:
        if kwargs["printSum"]:
            print("The total sum is %0.2f" % valueSum)
    
    if "subtractAmount" in kwargs:
        valueSum -= kwargs["subtractAmount"] 
    return valueSum    

In [7]:
sumValues(1, 2, 3, 4)

10

In [8]:
sumValues(1, 2, 3, 4, printSum=True)

The total sum is 10.00


10

In [9]:
sumValues(1, 2, 3, 4, printSum=True, subtractAmount=3)

The total sum is 10.00


7

## Use None and docstrings to speciffy dynamic default arguments
Also, write docstrings for every class, function and module

In [13]:
from datetime import datetime
import time

def log(message, when=None):
    """ Log a message with a timestamp
    
    Args:
        message: Message to print
        when: datetime when message occurred. Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print("%s : %s" % (when, message))

In [14]:
log("Hi there!")
time.sleep(0.1)
log("Hi again!")

2018-04-26 18:35:30.035000 : Hi there!
2018-04-26 18:35:30.136000 : Hi again!


Using None for default arguments is especially important when the arguments are mutable. In this example, the default dict will be shared by all calls to the function:

In [35]:
def getDict(default = {}):
    return default

In [38]:
d = getDict()
d

{}

In [39]:
d["this"] = 1

In [43]:
e = getDict()
e # modifying d has influenced the default return value of e

{'this': 1}

Use None as the default value for keyword argument and document.

## Enforce clarity with keyword only arguments